In [ ]:
import pandas as pd
from ipyfilechooser import FileChooser
from IPython.display import display
import os
from datetime import time

# Étape 1 : Fonction de prétraitement du fichier original
def preprocess_file(file_path):
    df = pd.read_csv(file_path)

    # Vérifie que la colonne de date/heure existe
    if 'MM:DD:YYYY hh:mm:ss' not in df.columns:
        raise ValueError("Colonne 'MM:DD:YYYY hh:mm:ss' absente du fichier.")

    # Séparer Date et Hour
    df[['Date', 'Hour']] = df['MM:DD:YYYY hh:mm:ss'].astype(str).str.split(' ', expand=True)
    df['Hour'] = pd.to_datetime(df['Hour'], format='%H:%M:%S', errors='coerce').dt.time
    df['Date'] = pd.to_datetime(df['Date'], format='%m:%d:%Y', errors='coerce').dt.date

    # Définir la période (Day/Night)
    def get_period(hour):
        if pd.isnull(hour):
            return None
        return 'Day' if time(7, 0, 1) <= hour <= time(19, 0, 0) else 'Night'
    df['Period'] = df['Hour'].apply(get_period)

    # Supprimer colonnes inutiles
    drop_cols = [
        'Library_Version', 'Session_type', 'Device_Number',
        'Battery_Voltage', 'Motor_Turns', 'FR'
    ]
    df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)

    # Normaliser les chaînes (minuscule + trim)
    if 'Event' in df.columns and 'Active_Poke' in df.columns:
        df['Event'] = df['Event'].astype(str).str.strip().str.lower()
        df['Active_Poke'] = df['Active_Poke'].astype(str).str.strip().str.lower()
    else:
        raise KeyError("Colonnes 'Event' et/ou 'Active_Poke' absentes du fichier.")

    return df

# Étape 2 : Calcul de l’impulsivité avec insertion des moyennes en haut
def calculer_impulsivite(df, original_file_path):
    impulsivite_scores = []
    compteur = 0
    en_attente_de_correct = False

    for _, row in df.iterrows():
        event = row['Event']
        active_poke = row['Active_Poke']

        if event == 'pellet':
            compteur = 0
            en_attente_de_correct = True
            continue

        if en_attente_de_correct:
            if event == active_poke:
                impulsivite_scores.append(compteur)
                compteur = 0
                en_attente_de_correct = False
            else:
                compteur += 1

    impulsivite_df = pd.DataFrame({
        'Essai': range(1, len(impulsivite_scores) + 1),
        'Erreurs avant poke correct': impulsivite_scores
    })

    # Moyennes
    moyenne_globale = impulsivite_df['Erreurs avant poke correct'].mean()
    moyenne_sans_zeros = impulsivite_df[impulsivite_df['Erreurs avant poke correct'] != 0]['Erreurs avant poke correct'].mean()

    # Insertion des moyennes en haut du fichier
    header_df = pd.DataFrame([
        {'Essai': 'Moyenne (avec 0)', 'Erreurs avant poke correct': f"{moyenne_globale:.2f}"},
        {'Essai': 'Moyenne (sans 0)', 'Erreurs avant poke correct': f"{moyenne_sans_zeros:.2f}"}
    ])

    final_df = pd.concat([header_df, impulsivite_df], ignore_index=True)

    # Sauvegarde
    base_name = os.path.splitext(os.path.basename(original_file_path))[0]
    folder = os.path.dirname(original_file_path)
    impulsivite_file_path = os.path.join(folder, f"{base_name}_impulsivite.csv")
    final_df.to_csv(impulsivite_file_path, index=False)

    # Affichage console
    print(f"\n📊 Moyenne AVEC les 0 : {moyenne_globale:.2f}")
    print(f"📊 Moyenne SANS les 0 : {moyenne_sans_zeros:.2f}")
    print(f"✅ Fichier sauvegardé ici : {impulsivite_file_path}")

    display(final_df.head(10))

# Étape 3 : Interface utilisateur
def choisir_fichier_et_lancer():
    def handle_file(chooser):
        selected_file = chooser.selected
        print(f"\n📂 Fichier sélectionné : {selected_file}")
        try:
            df = preprocess_file(selected_file)
            calculer_impulsivite(df, selected_file)
        except Exception as e:
            print(f"❌ Erreur lors du traitement : {e}")

    file_chooser = FileChooser(".", title="<b>Sélectionnez le fichier CSV</b>")
    display(file_chooser)
    file_chooser.register_callback(handle_file)

# Lancer l’interface
choisir_fichier_et_lancer()


FileChooser(path='C:\Users\AudreyHay\Documents\Carla\Visual Code\HayLabAnalysis\python', filename='', title='<…

,Essai,Erreurs avant poke correct
0,Moyenne (avec 0),0.61
1,Moyenne (sans 0),2.38
2,1,0
3,2,0
4,3,0
5,4,11
6,5,3
7,6,0
8,7,0
9,8,17


,Essai,Erreurs avant poke correct
0,Moyenne (avec 0),2.43
1,Moyenne (sans 0),4.56
2,1,2
3,2,6
4,3,6
5,4,1
6,5,0
7,6,0
8,7,0
9,8,14


,Essai,Erreurs avant poke correct
0,Moyenne (avec 0),1.25
1,Moyenne (sans 0),2.96
2,1,0
3,2,1
4,3,7
5,4,0
6,5,3
7,6,0
8,7,0
9,8,5


,Essai,Erreurs avant poke correct
0,Moyenne (avec 0),1.25
1,Moyenne (sans 0),3.39
2,1,0
3,2,0
4,3,1
5,4,0
6,5,0
7,6,2
8,7,0
9,8,0


: 